## 1. load packages and data

In [73]:
import pandas as pd
import numpy as np
from nilearn import datasets, image, plotting, surface
import nibabel as nib
from scipy import stats
from collections import Counter
import os
from collections import Counter
import csv

datasets_path = "D:\\code\\SC_ADHD\\datasets\\ABCD\\task-fMRI"
src_activation_path = os.path.join(datasets_path, "Destrieux")
surface_path = os.path.join(datasets_path, "surface")

In [74]:
destrieux_left_file = 'tpl-fsaverage_hemi-L_den-164k_atlas-Destrieux2009_dseg.label.gii'
destrieux_right_file = 'tpl-fsaverage_hemi-R_den-164k_atlas-Destrieux2009_dseg.label.gii'
yeo17_left_file = 'tpl-fsaverage_hemi-L_den-164k_atlas-Yeo2011_seg-17n_dseg.label.gii'
yeo17_right_file = 'tpl-fsaverage_hemi-R_den-164k_atlas-Yeo2011_seg-17n_dseg.label.gii'

def load_gii_labels(file_path):
    gii_data = nib.load(file_path).darrays[0].data
    return gii_data

In [75]:
destrieux_left_labels = load_gii_labels(os.path.join(surface_path, destrieux_left_file))
destrieux_right_labels = load_gii_labels(os.path.join(surface_path, destrieux_right_file))
yeo17_left_labels = load_gii_labels(os.path.join(surface_path, yeo17_left_file))
yeo17_right_labels = load_gii_labels(os.path.join(surface_path, yeo17_right_file))
destrieux_left_labels

array([29, 27,  8, ..., 62, 62, 62], shape=(163842,), dtype=int32)

In [76]:
print(len(destrieux_right_labels))
print(np.unique(yeo17_left_labels))

163842
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]


### fsaverage: Destrieux
1. label(L/R): 0~75
2. 42 is the Medial_wall

## 2. assign Destrieux label to Yeo17

In [77]:
destreux_left_mapping_list = []
destrieux_left_region_ids = np.unique(destrieux_left_labels)
destrieux_left_region_ids = destrieux_left_region_ids[(destrieux_left_region_ids != 0) & (destrieux_left_region_ids != 42)]

for idx in destrieux_left_region_ids:
    region_mask = (destrieux_left_labels == idx)
    region_yeo17_labels = yeo17_left_labels[region_mask]
    mode_result = stats.mode(region_yeo17_labels, keepdims=True)

    Yeo17_label = mode_result.mode[0]  # 取众数值

    update_idx = len(destreux_left_mapping_list) + 1
    destreux_left_mapping_list.append({
        'fsaverage_ID': int(idx),
        'Destrieux_ID': int(update_idx),
        'Yeo17_ID': int(Yeo17_label)
    })

destrieux_left_mapping_df = pd.DataFrame(destreux_left_mapping_list)
destrieux_left_mapping_df


,fsaverage_ID,Destrieux_ID,Yeo17_ID
0,1,1,13
1,2,2,1
2,3,3,3
3,4,4,4
4,5,5,16
...,...,...,...
69,71,70,10
70,72,71,16
71,73,72,17
72,74,73,17


In [78]:
destreux_right_mapping_list = []
destrieux_right_region_ids = np.unique(destrieux_right_labels)
destrieux_right_region_ids = destrieux_right_region_ids[
    (destrieux_right_region_ids != 0) & (destrieux_right_region_ids != 42)]

for idx in destrieux_right_region_ids:
    region_mask = (destrieux_right_labels == idx)
    region_yeo17_labels = yeo17_right_labels[region_mask]
    mode_result = stats.mode(region_yeo17_labels, keepdims=True)

    Yeo17_label = mode_result.mode[0]  # 取众数值

    update_idx = len(destreux_right_mapping_list) + 75
    destreux_right_mapping_list.append({
        'fsaverage_ID': int(idx),
        'Destrieux_ID': int(update_idx),
        'Yeo17_ID': int(Yeo17_label)
    })

destrieux_right_mapping_df = pd.DataFrame(destreux_right_mapping_list)
destrieux_right_mapping_df

,fsaverage_ID,Destrieux_ID,Yeo17_ID
0,1,75,13
1,2,76,1
2,3,77,3
3,4,78,4
4,5,79,10
...,...,...,...
69,71,144,16
70,72,145,16
71,73,146,13
72,74,147,14


In [79]:
destrieux_mapping_df = pd.concat([destrieux_left_mapping_df, destrieux_right_mapping_df])

## 3. count the vertices number

In [80]:
yeo17_vertices_count = {i: 0 for i in range(1, 18)}  # key: 1~17

destrieux_left_to_yeo17 = {}
for _, item in destrieux_left_mapping_df.iterrows():
    destrieux_left_to_yeo17[item['fsaverage_ID']] = item['Yeo17_ID']


for destrieux_id in destrieux_left_to_yeo17:
    left_region_mask = (destrieux_left_labels == destrieux_id)
    vertices_count_in_region = np.sum(left_region_mask)
    print(destrieux_id, vertices_count_in_region)

    yeo17_label = destrieux_left_to_yeo17[destrieux_id]

    if yeo17_label >= 1 and yeo17_label <= 17:
        yeo17_vertices_count[yeo17_label] += vertices_count_in_region


destrieux_right_to_yeo17 = {}
for _, item in destrieux_right_mapping_df.iterrows():
    destrieux_right_to_yeo17[item['fsaverage_ID']] = item['Yeo17_ID']


for destrieux_id in destrieux_right_to_yeo17:
    right_region_mask = (destrieux_right_labels == destrieux_id)
    vertices_count_in_region = np.sum(right_region_mask)

    yeo17_label = destrieux_right_to_yeo17[destrieux_id]

    if yeo17_label >= 1 and yeo17_label <= 17:
        yeo17_vertices_count[yeo17_label] += vertices_count_in_region


df_yeo17_volume = pd.DataFrame({
    'Yeo17_Network': list(yeo17_vertices_count.keys()),
    'vertices_Count': list(yeo17_vertices_count.values())
})

df_yeo17_volume = df_yeo17_volume.sort_values('Yeo17_Network').reset_index(drop=True)
Yeo17_assigned_volume_file = 'yeo17_network_vertices_count_from_destrieux.csv'
df_yeo17_volume.to_csv(os.path.join(datasets_path, "atlas mapping", Yeo17_assigned_volume_file), index=False)
df_yeo17_volume

1 932
2 1277
3 2272
4 2607
5 627
6 2171
7 1957
8 2386
9 1098
10 520
11 1430
12 1800
13 453
14 1005
15 4252
16 8394
17 769
18 924
19 2098
20 1365
21 1701
22 2483
23 2416
24 2520
25 3108
26 4030
27 4220
28 3509
29 3876
30 3540
31 1197
32 240
33 716
34 2784
35 851
36 1454
37 2536
38 2956
39 374
40 496
41 2018
43 2028
44 1661
45 3085
46 4914
47 1920
48 889
49 2463
50 3091
51 1151
52 323
53 2174
54 1376
55 3031
56 435
57 4409
58 951
59 1243
60 818
61 897
62 2354
63 356
64 822
65 1104
66 2220
67 2337
68 4447
69 1881
70 1846
71 742
72 1359
73 1111
74 6591
75 564


,Yeo17_Network,vertices_Count
0,1,22448
1,2,20843
2,3,30985
3,4,14089
4,5,11705
5,6,15712
6,7,32996
7,8,9365
8,9,16760
9,10,13774


In [81]:
# Destrieux atlas
Destrieux_vertices_count = {i: 0 for i in range(1, 149)}

destrieux_left_label_list = destrieux_left_mapping_df['Destrieux_ID']
for destrieux_id in destrieux_left_label_list:
    fsavg_id = destrieux_left_mapping_df[destrieux_left_mapping_df['Destrieux_ID'] == destrieux_id]['fsaverage_ID'].iloc[0]
    # print(fsavg_id)
    left_region_mask = (destrieux_left_labels == fsavg_id)
    vertices_count_in_region = np.sum(left_region_mask)

    Destrieux_vertices_count[destrieux_id] += vertices_count_in_region

destrieux_right_label_list = destrieux_right_mapping_df['Destrieux_ID']
for destrieux_id in destrieux_right_label_list:
    fsavg_id = destrieux_right_mapping_df[destrieux_right_mapping_df['Destrieux_ID'] == destrieux_id]['fsaverage_ID'].iloc[0]
    # print(fsavg_id)
    right_region_mask = (destrieux_right_labels == fsavg_id)
    vertices_count_in_region = np.sum(right_region_mask)

    Destrieux_vertices_count[destrieux_id] += vertices_count_in_region

destrieux_volume = pd.DataFrame({
    'Destrieux_ID': list(Destrieux_vertices_count.keys()),
    'vertices_Count': list(Destrieux_vertices_count.values())
})

destrieux_volume = destrieux_volume.sort_values('Destrieux_ID').reset_index(drop=True)
Destrieux_assigned_volume_file = 'destrieux_network_vertices_count.csv'
destrieux_volume.to_csv(os.path.join(datasets_path, "atlas mapping", Destrieux_assigned_volume_file), index=False)
# Destrieux_vertices_count
destrieux_volume

,Destrieux_ID,vertices_Count
0,1,932
1,2,1277
2,3,2272
3,4,2607
4,5,627
...,...,...
143,144,433
144,145,1808
145,146,1056
146,147,7803


In [83]:
# build the atlas index-name mapping table
src_atlas_mapping_df = pd.read_csv(os.path.join(datasets_path, "atlas mapping", "Destrieux2Yeo17_reflect.csv"))

src_atlas_mapping_df.drop("Destrieux_ID", axis=1, inplace=True)
src_atlas_mapping_df.drop("Yeo17_ID", axis=1, inplace=True)
src_atlas_mapping_df.rename(columns={"index": "Destrieux_ID"}, inplace=True)
Destrieux_Yeo17_mapping_df = pd.merge(destrieux_mapping_df, src_atlas_mapping_df, on='Destrieux_ID', how='left')
Destrieux_Yeo17_mapping_df['Yeo17 Label'] = "17Networks_" + Destrieux_Yeo17_mapping_df['Yeo17_ID'].astype(str)
yeo17_label_name = {
    "17Networks_1": "VisCent",
    "17Networks_2": "VisPeri",
    "17Networks_3": "SomMotA",
    "17Networks_4": "SomMotB",
    "17Networks_5": "DorsAttnA",
    "17Networks_6": "DorsAttnB",
    "17Networks_7": "SalVentAttnA",
    "17Networks_8": "SalVentAttnB",
    "17Networks_9": "Limbic",
    "17Networks_10": "Limbic",
    "17Networks_11": "ContC",
    "17Networks_12": "ContA",
    "17Networks_13": "ContB",
    "17Networks_14": "TempPar",
    "17Networks_15": "DefaultC",
    "17Networks_16": "DefaultA",
    "17Networks_17": "DefaultB"
}
Destrieux_Yeo17_mapping_df["Yeo17 atlas label"] = Destrieux_Yeo17_mapping_df['Yeo17 Label'].map(yeo17_label_name)
Destrieux_Yeo17_mapping_df = pd.merge(Destrieux_Yeo17_mapping_df, destrieux_volume, on='Destrieux_ID', how='left')
Destrieux_Yeo17_mapping_df.to_csv(os.path.join(datasets_path, "atlas mapping", "Destrieux_Yeo17_mapping_table.csv"))
Destrieux_Yeo17_mapping_df

,fsaverage_ID,Destrieux_ID,Yeo17_ID,Destrieux atlas label,Yeo17 Label,Yeo17 atlas label,vertices_Count
0,1,1,13,L G_and_S_frontomargin,17Networks_13,ContB,932
1,2,2,1,L G_and_S_occipital_inf,17Networks_1,VisCent,1277
2,3,3,3,L G_and_S_paracentral,17Networks_3,SomMotA,2272
3,4,4,4,L G_and_S_subcentral,17Networks_4,SomMotB,2607
4,5,5,16,L G_and_S_transv_frontopol,17Networks_16,DefaultA,627
...,...,...,...,...,...,...,...
143,71,144,16,R S_suborbital,17Networks_16,DefaultA,433
144,72,145,16,R S_subparietal,17Networks_16,DefaultA,1808
145,73,146,13,R S_temporal_inf,17Networks_13,ContB,1056
146,74,147,14,R S_temporal_sup,17Networks_14,TempPar,7803


## 4. mapping activation table from Destrieux to  Yeo17

In [84]:
sst_folder_path = os.path.join(datasets_path, "sst")

# parcel size: df_yeo17_volume

for file in os.listdir(sst_folder_path):
    new_file = "yeo17_" + file
    filepath = os.path.join(sst_folder_path, file)
    new_file_path = os.path.join(sst_folder_path, new_file)
    if os.path.isfile(filepath) and not file.startswith("yeo17"):
        sst_table = pd.read_csv(filepath)
        columns = sst_table.columns
        basic_columns = ["src_subject_id", "eventname"]
        parcel_columns = [x for x in columns if x not in basic_columns]
        # result dataframe
        result_df = sst_table.drop(columns=parcel_columns).copy()
        yeo17_labels = sorted(Destrieux_Yeo17_mapping_df['Yeo17 Label'].unique())

        for yeo_label in yeo17_labels:
            # get Destrieux id according the yeo17
            yeo_id = yeo_label.split("_")[-1]
            mapping_subset = Destrieux_Yeo17_mapping_df[
                Destrieux_Yeo17_mapping_df['Yeo17 Label'] == yeo_label
            ]

            weighted_sum = pd.Series(0, index=sst_table.index)


            weight_scale = 0
            for _, row in mapping_subset.iterrows():
                destrieux_label = row['Destrieux atlas label']
                weight = row['vertices_Count']
                weight_scale += weight
                if destrieux_label in parcel_columns:
                    weighted_sum += sst_table[destrieux_label] * weight

            weighted_sum /= weight_scale
            result_df[yeo_label] = weighted_sum
        result_df.to_csv(new_file_path, index=False)
        # print(result_df.head())
        # break


In [85]:
nback_folder_path = os.path.join(datasets_path, "nback")

# parcel size: df_yeo17_volume

for file in os.listdir(nback_folder_path):
    new_file = "yeo17_" + file
    filepath = os.path.join(nback_folder_path, file)
    new_file_path = os.path.join(nback_folder_path, new_file)
    if os.path.isfile(filepath) and not file.startswith("yeo17"):
        nback_table = pd.read_csv(filepath)
        columns = nback_table.columns
        basic_columns = ["src_subject_id", "eventname"]
        parcel_columns = [x for x in columns if x not in basic_columns]
        # result dataframe
        result_df = nback_table.drop(columns=parcel_columns).copy()
        yeo17_labels = sorted(Destrieux_Yeo17_mapping_df['Yeo17 Label'].unique())

        for yeo_label in yeo17_labels:
            # get Destrieux id according the yeo17
            yeo_id = yeo_label.split("_")[-1]
            mapping_subset = Destrieux_Yeo17_mapping_df[
                Destrieux_Yeo17_mapping_df['Yeo17 Label'] == yeo_label
            ]

            weighted_sum = pd.Series(0, index=nback_table.index)


            weight_scale = 0
            for _, row in mapping_subset.iterrows():
                destrieux_label = row['Destrieux atlas label']
                weight = row['vertices_Count']
                weight_scale += weight
                if destrieux_label in parcel_columns:
                    weighted_sum += nback_table[destrieux_label] * weight

            weighted_sum /= weight_scale
            result_df[yeo_label] = weighted_sum
        result_df.to_csv(new_file_path, index=False)
        # print(result_df.head())
        # break
